<a href="https://colab.research.google.com/github/Nithyasri-kc/Highlighting-face-in-an-image-using-CNN/blob/main/Visage_vibe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gradio as gr


base_model = keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

def create_model():
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cascade_path)

def hex_to_bgr(hex_color):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return rgb[::-1]

def detect_faces(image, confidence_threshold=0.5, box_color='#00FF00', show_confidence=True):
    if image is None:
        return None

    image = np.array(image)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    bgr_color = hex_to_bgr(box_color)


    for (x, y, w, h) in faces:

        face_roi = cv2.resize(image[y:y+h, x:x+w], (224, 224))
        face_roi = np.array(face_roi) / 255.0
        face_roi = np.expand_dims(face_roi, axis=0)

        prediction = model.predict(face_roi)

        if prediction[0][0] > confidence_threshold:
            confidence = prediction[0][0]

            cv2.rectangle(image, (x, y), (x + w, y + h), bgr_color, 2)


            if show_confidence:
                cv2.putText(image, f'{confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr_color, 2)


    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def clear_inputs():
    return None, 0.5, "#00FF00", True


with gr.Blocks() as demo:
    gr.Markdown("# VisageVibe (Interactive Face Detection)")
    gr.Markdown("Upload an image to detect faces with customizable options.")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Image")
            confidence_slider = gr.Slider(0.0, 1.0, value=0.5, label="Confidence Threshold")
            color_picker = gr.ColorPicker(value="#00FF00", label="Box Color")
            show_confidence = gr.Checkbox(value=True, label="Show Confidence Score")

            with gr.Row():
                submit_button = gr.Button("Submit")
                clear_button = gr.Button("Clear")

        with gr.Column():
            output_image = gr.Image(type="numpy", label="Output Image")

    submit_button.click(
        fn=detect_faces,
        inputs=[image_input, confidence_slider, color_picker, show_confidence],
        outputs=output_image
    )

    clear_button.click(
        fn=clear_inputs,
        inputs=[],
        outputs=[image_input, confidence_slider, color_picker, show_confidence]
    )

    gr.Markdown("## Settings")
    gr.Markdown("""
    - *Confidence Threshold*: Adjust the minimum confidence level to display a detected face.
    - *Box Color*: Choose the color of the rectangle drawn around detected faces.
    - *Show Confidence Score*: Toggle the display of confidence scores above detected faces.
    """)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5d7cb00779ed68d8bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
